# Sistema de Recomendación

In [1]:
# imports
import pandas as pd
from funciones_scraping import get_languages

df = pd.read_csv('../output/df-limpio-iter3.csv', index_col=0)
df.shape

(11428, 55)

In [2]:
df.head()
df['c++'].value_counts()

0.0    11428
Name: c++, dtype: int64

In [3]:
languages = get_languages()
languages

{'bf (beta)',
 'c',
 'c#',
 'c++',
 'clojure',
 'coffeescript',
 'crystal',
 'dart',
 'elixir',
 'elm (beta)',
 'erlang (beta)',
 'f#',
 'fortran (beta)',
 'go',
 'groovy (beta)',
 'haskell',
 'java',
 'javascript',
 'julia (beta)',
 'kotlin (beta)',
 'lua (beta)',
 'nasm (beta)',
 'nim (beta)',
 'objective-c (beta)',
 'ocaml (beta)',
 'php',
 'powershell (beta)',
 'purescript (beta)',
 'python',
 'r (beta)',
 'ruby',
 'rust',
 'scala (beta)',
 'shell',
 'solidity (beta)',
 'sql',
 'swift',
 'typescript'}

In [4]:
def normalize_columns(data, columns): 
    for col in columns: 
        if data[col].max() > 0: 
            data[col] = data[col]/data[col].max()
    return data

df_norm = normalize_columns(df, languages)
df_norm.head()

,allies,bf (beta),c,c#,c++,clojure,coffeescript,collections,comments,crystal,...,sql,swift,total completed kata,total languages trained,translations,typescript,username,github,linkedin,time
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,45.0,2.0,0.0,0.0,brkiesel,NaN,NaN,1400
1,374.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,70.0,4.0,0.0,0.0,Em-Ant,https://github.com/Em-Ant,NaN,1096
2,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,50.0,2.0,0.0,0.0,Mikeysax,https://github.com/Mikeysax,NaN,731
3,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,139.0,3.0,0.0,0.0,kimiamania,https://github.com/kimiamania,https://www.linkedin.com/in/rezhajulio,823
4,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,17.0,2.0,0.0,0.0,Daigle,NaN,NaN,30


In [5]:
df_norm['c++'].value_counts()

0.0    11428
Name: c++, dtype: int64

In [6]:
df_norm.describe()

,allies,bf (beta),c,c#,c++,clojure,coffeescript,collections,comments,crystal,...,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,time
count,11428.000000,11428.0,11428.000000,11428.0,11428.0,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,...,11428.0,11428.000000,11428.0,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000
mean,93.289902,0.0,0.001295,0.0,0.0,0.001161,0.000957,0.143857,13.353255,0.000312,...,0.0,0.000551,0.0,0.009283,0.000642,102.076041,2.412496,1.488799,0.001119,496.643420
std,232.583439,0.0,0.019172,0.0,0.0,0.017324,0.014934,0.843151,173.395939,0.011377,...,0.0,0.012697,0.0,0.056273,0.015932,289.016703,2.597626,39.647146,0.020352,461.830188
min,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,91.000000
50%,15.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,28.000000,2.000000,0.000000,0.000000,395.000000
75%,72.250000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,76.000000,3.000000,0.000000,0.000000,791.000000
max,10610.000000,0.0,1.000000,0.0,0.0,1.000000,1.000000,29.000000,8962.000000,1.000000,...,0.0,1.000000,0.0,1.000000,1.000000,7811.000000,38.000000,2927.000000,1.000000,2252.000000


In [27]:
def ask_languages(languages=languages): 
    while True: 
        try: 
            res = input('Introduce los tags(lenguajes) separados por coma:').lower().split(',')
            res = [r.strip() for r in res]
            break
        except: 
            print('Fallo en la captura de requisitos. Inténtalo de nuevo')
    return res
    

In [28]:
ask_languages()

Introduce los tags(lenguajes) separados por coma:python, c


['python', 'c']

In [32]:
def distacia_euclidea(v1, v2): 
    return sum([(x1-x2)**2 for x1, x2 in zip(v1, v2)])**0.5

# distacia_euclidea([3, 4], [0, 0])

5.0

In [ ]:
def get_recomendaciones(data, skills=[]): 
    res = []
    for i in range(5): 
        res.append(data.loc[i])
    return res
        
        

In [40]:
for col, value in df_norm.loc[4, :].items(): 
    print (col, value)

allies 13.0
bf (beta) 0.0
c 0.0
c# 0.0
c++ 0.0
clojure 0.0
coffeescript 0.0
collections 0.0
comments 0.0
crystal 0.0
dart 0.0
elixir 0.0
elm (beta) 0.0
erlang (beta) 0.0
f# 0.0
followers 13.0
following 13.0
fortran (beta) 0.0
go 0.0
groovy (beta) 0.0
haskell 0.0
honor 127.0
java 7.921733275240622e-05
javascript 0.0013666442888654454
julia (beta) 0.0
kotlin (beta) 0.0
kumite 0.0
leaderboard position 0.0
lua (beta) 0.0
most recent R
nasm (beta) 0.0
nim (beta) 0.0
objective-c (beta) 0.0
ocaml (beta) 0.0
php 0.0
powershell (beta) 0.0
purescript (beta) 0.0
python 0.0
r (beta) 0.0
rank 6.0
ruby 0.0
rust 0.0
scala (beta) 0.0
shell 0.0
solidity (beta) 0.0
sql 0.0
swift 0.0
total completed kata 17.0
total languages trained 2.0
translations 0.0
typescript 0.0
username Daigle
github nan
linkedin nan
time 30
